In [176]:
## Some interesting things from VicRoads Crash Stats

#### Import some data

In [218]:
import pandas as pd

df_accident = pandas.read_csv(
    'vicroads_data/ACCIDENT/ACCIDENT.csv', # file name and path
    sep=',', # separated by commas
    index_col=0#, # first column should be primary key
   # parse_dates=[1,2] # next two columns represent dates and times
)

In [219]:
# data read okay?
print(df_accident.head(3))

             ACCIDENTDATE ACCIDENTTIME  ACCIDENT_TYPE  \
ACCIDENT_NO                                             
T20060000010   13/01/2006     12.42.00              1   
T20060000018   13/01/2006     19.10.00              1   
T20060000022   14/01/2006     12.10.00              7   

                          Accident Type Desc  DAY_OF_WEEK  \
ACCIDENT_NO                                                 
T20060000010          Collision with vehicle            6   
T20060000018          Collision with vehicle            6   
T20060000022  Fall from or in moving vehicle            7   

             Day Week Description  DCA_CODE  \
ACCIDENT_NO                                   
T20060000010               Friday       113   
T20060000018               Friday       113   
T20060000022             Saturday       190   

                                             DCA Description DIRECTORY  \
ACCIDENT_NO                                                              
T20060000010  RIGHT NEAR

## charts

#### Include Libraries

In [221]:
import pandas as pd
import numpy as np
import plotly.offline as py
from plotly.offline import *
import plotly.graph_objs as go

init_notebook_mode(connected=True) # render plotly charts in the notebook on the fly

#### data prep

Manipulate data: create a copy of df_accident, add columns for:
* Year
* Month

Filter out 2006 and 2017, as they are incomplete in this data

In [238]:
# Manipulate data
df_accident_mod = df_accident
#print(df_accident_mod.head(3))

## DATES

# make a string representing date time
df_accident_mod['date'] = df_accident_mod['ACCIDENTDATE'] + ' ' + df_accident_mod['ACCIDENTTIME']

# convert to datetime
df_accident_mod['date'] = pd.to_datetime(df_accident_mod['date'], format="%d/%m/%Y %H.%M.%S")

# make some useful date fields
df_accident_mod['year'] = pd.DatetimeIndex(df_accident_mod['date']).year
df_accident_mod['month'] = pd.DatetimeIndex(df_accident_mod['date']).month
df_accident_mod['hour'] = pd.DatetimeIndex(df_accident_mod['date']).hour

#print(df_accident_mod.head(3))

## FILTER

# remove < 2007 and > 2016, as 2006 and 2017 are incomplete years
df_accident_mod = df_accident_mod[df_accident_mod['year'] >= 2007]
df_accident_mod = df_accident_mod[df_accident_mod['year'] <= 2016]

# remove speed limits > 110 km/h - 777, 888, and 999 are used to code:
# 777 = other
# 888 = camping grounds or off road
# 999 = not known

df_accident_mod = df_accident_mod[df_accident_mod['SPEED_ZONE'] <= 110]

print(df_accident_mod.head(3))

             ACCIDENTDATE ACCIDENTTIME  ACCIDENT_TYPE  \
ACCIDENT_NO                                             
T20070000004    1/01/2007     02.55.00              4   
T20070000008    1/01/2007     03.59.00              1   
T20070000009    1/01/2007     03.30.00              4   

                         Accident Type Desc  DAY_OF_WEEK Day Week Description  \
ACCIDENT_NO                                                                     
T20070000004  Collision with a fixed object            2               Monday   
T20070000008         Collision with vehicle            2               Monday   
T20070000009  Collision with a fixed object            2               Monday   

              DCA_CODE                                    DCA Description  \
ACCIDENT_NO                                                                 
T20070000004       171  LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...   
T20070000008       140                  U TURN                              


#### Make some charts

* run group bys in pandas
* configure chart layout
* display

In [181]:
accidents_by_year =  df_accident_mod.groupby([ 'year']).size().reset_index(name="cnt")
print(accidents_by_year.head(3))

   year    cnt
0  2007  13483
1  2008  14036
2  2009  13704


In [182]:
yearSeries = go.Bar(
    x = accidents_by_year['year'],
    y = accidents_by_year['cnt'],
    #mode = 'scatter',
    name = 'Crashes per Year',
    marker = dict(
        color = '#ee2737'
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents per Year',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 1, # label every 1 bars
        title = 'Year',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        )
    )
)

data = [yearSeries] # can be multiple series

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='figureYear')

#### Accidents by Month

# TODO
* Control for days in month
* label months Jan, Feb, etc

In [183]:
accidents_by_month =  df_accident_mod.groupby([ 'month']).size().reset_index(name="cnt")
print(accidents_by_month.head(3))

monthSeries = go.Bar(
    x = accidents_by_month['cnt'],
    y = accidents_by_month['month'],
    
    #mode = 'scatter',
    name = 'Crashes per Month',
    marker = dict(
        color = '#ee2737'
    ),
    orientation = 'h'
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents per Month',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 1, # label every 1 bars
        title = 'Month',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        )
    )
)

data = [monthSeries] # can be multiple series

figureMonth = go.Figure(data=data, layout=layout)

py.iplot(figureMonth, filename='figureMonth')

   month    cnt
0      1  11430
1      2  11170
2      3  12272


#### Accidents by Speed Zone

In [239]:
# 777, 888, and 999 exist in the SPEED_ZONE column
# remove >110


accidents_by_speed =  df_accident_mod.groupby([ 'SPEED_ZONE']).size().reset_index(name="cnt")
print(accidents_by_speed.head(3))

speedLimitZoneSeries = go.Bar(
    y = accidents_by_speed['cnt'],
    x = accidents_by_speed['SPEED_ZONE'],
    name = 'Num. Crashes',
    marker = dict(
        color = '#ee2737'
    )
)

layout = go.Layout(
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 1 bars
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        )
    )
)

data = [speedLimitZoneSeries] # can be multiple series

figurespeedLimitZone = go.Figure(data=data, layout=layout)

py.iplot(figurespeedLimitZone, filename='figurespeedLimitZone')

   SPEED_ZONE    cnt
0          30    170
1          40   5388
2          50  25395


#### All accidents vs Fatal accidents

In [240]:
# group by severity and speed zone
accidents_by_speed_by_severity =  df_accident_mod.groupby(['SPEED_ZONE','SEVERITY']).size().reset_index(name="cnt")
print(accidents_by_speed_by_severity.head(3))

# only keep severe
accidents_by_speed_severity_1 = accidents_by_speed_by_severity[accidents_by_speed_by_severity['SEVERITY'] == 1]
print(accidents_by_speed_severity_1.head(3))

speedLimitZoneSeriesSevere = go.Bar(
    y = accidents_by_speed_severity_1['cnt'],
    x = accidents_by_speed_severity_1['SPEED_ZONE'],
    name = 'Severe Crashes by Speed Limit Zone',
    marker = dict(
        color = '#0077c8'
    )
)

layout = go.Layout(
    #barmode = 'stack',
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 1 bars
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Count',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        )
    )
)

data = [speedLimitZoneSeries, speedLimitZoneSeriesSevere] # can be multiple series

figurespeedLimitZoneSevere = go.Figure(data=data, layout=layout)

py.iplot(figurespeedLimitZoneSevere, filename='figurespeedLimitZoneSevere')

   SPEED_ZONE  SEVERITY  cnt
0          30         1    1
1          30         2   52
2          30         3  117
   SPEED_ZONE  SEVERITY  cnt
0          30         1    1
3          40         1   24
6          50         1  222


#### % Crashes Lethal

This snippet of code:
* Groups by both Speed Zone and Severity
* Applys a function to calculate a fields percentage of the first group by field (speed zone), so all speed zone 30 sum to 100, all speed zone 40 sum to 100, etc
* filter for just severe crashes (severity = 1), indicating someone died

In [241]:
# group by severity and speed zone
accidents_by_speed_by_severity =  df_accident_mod.groupby(['SPEED_ZONE','SEVERITY']).size()
# divide by the first level you grouped by, multiply by 100, to end up with each severitys % of speed zone crashes
accidents_by_speed_by_severity_pc = accidents_by_speed_by_severity.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

# reset index and name the new column count
accidents_by_speed_by_severity_pc = accidents_by_speed_by_severity_pc.reset_index(name="count")

# filter for just severe
accidents_by_speed_severe_pc = accidents_by_speed_by_severity_pc[accidents_by_speed_by_severity_pc['SEVERITY'] == 1]

print(accidents_by_speed_severe_pc)


    SPEED_ZONE  SEVERITY     count
0           30         1  0.588235
3           40         1  0.445434
6           50         1  0.874188
9           60         1  1.010531
13          70         1  1.740580
16          75         1  3.571429
19          80         1  2.114687
22          90         1  3.770950
25         100         1  5.271744
28         110         1  6.228374


In [243]:

speedLimitZoneSeriesSeverePc = go.Scatter(
    y = accidents_by_speed_severe_pc['count'],
    x = accidents_by_speed_severe_pc['SPEED_ZONE'],
    name = '% Crashes Lethal',
    marker = dict(
        color = '#0077c8'
    ),
    yaxis='y2'
)

layout = go.Layout(
    barmode = 'group',
    title = 'VicRoads Accident - Accidents by Speed Limit Zone',
    titlefont=dict(
            family='Open Sans',
            size=22#,
            #color='#7f7f7f'
    ),
    xaxis=dict(
        tickangle = -45, # angle lables at 45 deg
        dtick = 10, # label every 10th number after 0
        title = 'Speed Limit Zone',
            titlefont=dict(
                family='Open Sans',
                size=16#,
                #color='#7f7f7f'
            )
    ),
    yaxis = dict(
        title = 'Num. Crashes',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        ),
        dtick=5000,
       # overlaying = 'y2',
        range=[0,65000]#,
        #rangemode = 'tozero' # start from zero
    ),
    yaxis2=dict(
        title='% Crashes Lethal',
        overlaying='y',
        side='right',
        titlefont=dict(
            family='Open Sans',
            size=16#,
            #color='#7f7f7f'
        ),
        range=[0,6.5],
        dtick = 1#, # label every 1
        #rangemode = 'tozero'
    )
)

data = [speedLimitZoneSeries, speedLimitZoneSeriesSeverePc] # can be multiple series

figurespeedLimitZoneSeverePc = go.Figure(data=data, layout=layout)

py.iplot(figurespeedLimitZoneSeverePc, filename='figurespeedLimitZoneSeverePc')

### Crashes by Hour

In [236]:
accidents_by_hour =  df_accident_mod.groupby(['hour']).size().reset_index(name="cnt")
print(accidents_by_hour.head(3))

   hour   cnt
0     0  2137
1     1  1814
2     2  1463


### Crashes by Population Estimates

### Group demo data by Age and Gender, make a line chart

#### Data Munging

In [187]:
# group by gender group by age, return count
age_gender =  df.groupby([ 'gender', 'age']).size().reset_index(name="count")
print(age_gender.head(3))

# make a list of 0 -> 100 for the x axis
ages = list(range(101))
ages_df = pandas.DataFrame(ages, columns = ['age']) # make that a data frame, give it the title Age
print(ages_df.head(3))

# make a data series for women
f = age_gender.loc[age_gender['gender'] == 'F'] # filter for sex = 'F'
f = f.drop('gender', axis=1) # drop gender column
female = ages_df.merge(f, left_on='age', right_on='age', how='left') # left join to ages to get all ages
female['count'].fillna(0, inplace=True) # replace NAN with 0
print(female.head(3))

# make a data series for men
m = age_gender.loc[age_gender['gender'] == 'M'] # filter for sex = 'F'
m = m.drop('gender', axis=1) # drop gender column
male = ages_df.merge(m, left_on='age', right_on='age', how='left') # left join to ages to get all ages
male['count'].fillna(0, inplace=True) # replace NAN with 0
print(male.head(3))

NameError: name 'df' is not defined

#### Make chart elements

In [ ]:
womenSeries = go.Scatter(
    x = female['age'],
    y = female['count'],
    mode = 'columns',
    name = 'Women'
)

menSeries = go.Scatter(
    x = male['age'],
    y = male['count'],
    mode = 'columns',
    name = 'Men'
)

layout = go.Layout(
    xaxis=dict(
        range=[18, 100]
    )
)

#### Make chart

In [ ]:
data = [womenSeries, menSeries]

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='line-mode')